<a href="https://colab.research.google.com/github/akbarjimi/BlossomNet/blob/main/SimpleNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Load Dataset 🔄

In [ ]:
import os
import numpy as np
import random
import csv
from typing import List
import math

In [ ]:
def load_dataset(file_path: str) -> List[List[float]]:
    """
    Loads the dataset from a CSV file.

    Parameters:
    file_path (str): Path to the dataset file.

    Returns:
    data (list): Loaded dataset in list format (could be a numpy array, etc.)
    """
    data = []
    with open(file_path , 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            data.append([float(feature) for feature in row[:-1]])
    return data

In [ ]:
def validate_dataset(dataset: List) -> bool:
    """
    Validates the structure and integrity of the dataset.

    Parameters:
    dataset (list): The loaded dataset.

    Returns:
    bool: True if dataset is valid, False otherwise.
    """
    if len(dataset) == 0:
        return False

    num_features = len(dataset[0])
    for row in dataset:
        if len(row) != num_features:
            return False

    return True

In [ ]:
def test_data_loading():
    """
    Tests if the dataset loading function works as expected.
    """
    try:
        file_path = '/content/drive/MyDrive/Colab Notebooks/First Neural Network/Iris.csv'
        dataset = load_dataset(file_path)
        assert validate_dataset(dataset), "Dataset validation failed."
        print("Data loaded and validated successfully.")
    except AssertionError as error:
        print(f"Test failed: {error}")
    except FileNotFoundError:
        print("File not found. Make sure the dataset file exists at the specified path.")
    except Exception as error:
        print(f"An error occurred: {error}")

# 2. Normalize the Data 🔄

In [ ]:
def transpose(data: List[List]) -> List[List]:
    """Transposes a list of lists.

    Args:
    data: A list of lists.

    Returns:
    A transposed list of lists.
    """

    return list(zip(*data))

In [ ]:
def min_max_normalizer(data: List[List[float]]) -> List[List[float]]:
    """
    Applies Min-Max scaling to the dataset, scaling features between 0 and 1.

    Parameters:
    data (list of lists): The dataset to be normalized.

    Returns:
    scaled_data (list of lists): Min-max normalized dataset.
    """

    transposed_data = transpose(data)

    min_vals = [min(col) for col in transposed_data]
    max_vals = [max(col) for col in transposed_data]

    scaled_data = []

    for row in data:
        try:
            scaled_row = [(val - min_val) / (max_val - min_val) if (max_val - min_val) != 0 else 0.0 for val, min_val,max_val in zip(row, min_vals, max_vals)]
            scaled_data.append(scaled_row)
        except ZeroDivisionError:
            row_index = data.index(row)
            print(f"ZeroDivisionError encountered in row {row_index}. Values: {row}, {min_vals}, {max_vals}")
            raise ZeroDivisionError

    return scaled_data

# 3. Define the Architecture 🏗

In [2]:
class Neuron:
    """A simple neuron class with type hints."""

    def __init__(self, weights: List[float], bias: float = None):
        """Initializes a neuron with given weights and bias.

        Args:
            weights: A list of weights for the neuron's inputs.
            bias: The bias term for the neuron.
        """
        self.weights = weights
        self.bias = bias if bias is not None else random.uniform(-0.1, 0.1)
        self.inputs = []  # To store inputs for backpropagation
        self.output = 0   # To store output for backpropagation

    def forward(self, inputs: List[float]) -> float:
        """Calculates the output of the neuron.

        Args:
            inputs: A list of inputs to the neuron.

        Returns:
            The output of the neuron.
        """
        self.inputs = inputs
        weighted_sum = sum([input_ * weight for input_, weight in zip(inputs, self.weights)])
        self.output = weighted_sum + self.bias
        return self.output

    def activation(self, output: float) -> float:
        """Applies an activation function to the neuron's output.

        Args:
            output: The output of the neuron.

        Returns:
            The activated output of the neuron.
        """
        return output

    def compute_gradient(self, delta: float) -> List[float]:
        """Calculates the gradient for the weights using the delta from the next layer.

        Args:
            delta: The error signal from the next layer.

        Returns:
            A list of gradients for the weights.
        """
        gradients = [delta * input_ for input_ in self.inputs]
        return gradients

    def update_weights(self, learning_rate: float, gradients: List[float]):
        """Updates the weights and bias of the neuron using the computed gradients.

        Args:
            learning_rate: The learning rate for weight updates.
            gradients: The computed gradients for each weight.
        """
        self.weights = [w - learning_rate * g for w, g in zip(self.weights, gradients)]
        self.bias -= learning_rate * gradients[-1]  # Updating bias with the last gradient

    def propagate_error_back(self) -> List[float]:
        """Propagates the error signal back to the previous layer.

        Returns:
            A list of error terms to propagate to the previous layer.
        """
        # Example: Assuming we are using the derivative of the sigmoid function for backpropagation
        error_signal = self.output * (1 - self.output)
        propagated_errors = [error_signal * weight for weight in self.weights]
        return propagated_errors

NameError: name 'List' is not defined

In [ ]:
class ActivationFunctions:
    """A utility class for activation functions."""

    @staticmethod
    def sigmoid(x: float) -> float:
        """Applies the sigmoid activation function.

        Args:
            x: The input value.

        Returns:
            The activated value.
        """
        return 1 / (1 + math.exp(-x))

    @staticmethod
    def relu(x:float) -> float:
        """Applies the rectified linear unit (ReLU) activation function.

        Args:
            x: The input value.

        Returns:
            The activated value.
        """
        return max(0,x)

    @staticmethod
    def softmax(x: List[float]) -> List[float]:
        """Applies the softmax activation function.

        Args:
            x: A list of input values.

        Returns:
            A list of normalized probabilities.
        """

        # returns the base e raised to the power of the given number
        exp_values = [math.exp(value) for value in x]
        sum_exp_values = sum(exp_values)
        return [exp_value / sum_exp_values for exp_value in exp_values]

In [ ]:
class Layer:
    """A layer in a neural network."""

    def __init__(self, neurons: List[Neuron], activation: ActivationFunctions):
        """Initializes a layer with the specified number of neurons and inputs.

        Args:
            neurons: The neurons in the layer.
            activation: The activation function to use for the neurons.
        """
        self.neurons = neurons
        self.activation = activation

    def forward(self, inputs: List[float]) -> List[float]:
        """Propagates inputs through the layer.

        Args:
            inputs: A list of inputs to the layer.

        Returns:
            A list of outputs from the neurons in the layer.
        """
        neuron_outputs = [neuron.forward(inputs) for neuron in self.neurons]
        # Apply the activation function after calculating the neuron outputs
        activated_outputs = [self.activation(output) for output in neuron_outputs]
        return activated_outputs

    def backward(self, delta: List[float], learning_rate: float) -> List[float]:
        """Performs the backward pass and updates weights and biases.

        Args:
            delta: The error term (gradient) from the next layer.
            learning_rate: The learning rate for weight updates.

        Returns:
            A list of propagated error terms to pass to the previous layer.
        """
        new_delta = []
        for i, neuron in enumerate(self.neurons):
            neuron_gradient = neuron.compute_gradient(delta[i])

            neuron.update_weights(learning_rate)

            new_delta.append(neuron.propagate_error_back())

        return new_delta

"""
random.seed(1)
# Create a layer with 3 neurons and 2 inputs
neurons = [Neuron([random.uniform(-1, 1) for _ in range(2)], random.uniform(-1, 1)) for _ in range(3)]
layer = Layer(neurons, ActivationFunctions().relu)

# Input data
inputs = [0.5, 0.8]

# Forward propagation
outputs = layer.forward(inputs)
print(outputs)  # Output: [0.57926035, 0.58828428, 0.59730822]
"""

[0.7178074611652016, 0, 0]


In [ ]:
class Network:
    """A neural network with multiple layers."""


    def __init__(self, layers: List[Layer], epochs: int, learning_rate: float):
        self.layers = layers
        self.learning_rate = learning_rate
        self.epochs = epochs

    def forward(self, inputs: List[float]) -> List[float]:
        """Propagates inputs through the entire network.

        Args:
            inputs: A list of inputs to the network.

        Returns:
            A list of outputs from the final layer.
        """
        outputs = inputs
        for layer in self.layers:
            outputs = layer.forward(outputs)
        return outputs

    def backward(self, targets: List[float], outputs: List[float]):
        """Performs backpropagation to update the network's weights and biases.

        Args:
            targets: A list of target outputs.
            outputs: A list of outputs from the final forward pass.
        """
        # Compute the gradient of the loss with respect to the output (initial delta)
        delta = self.loss_derivative(outputs, targets)

        # Iterate backward over the layers to propagate the error and update weights
        for layer in reversed(self.layers):
            delta = layer.backward(delta, self.learning_rate)

    def compute_loss(self, predicted: List[float], actual: List[float]) -> float:
        """Calculates the loss for the predictions."""
        return LossFunction.cross_entropy(predicted, actual)

    def loss_derivative(self, outputs: List[float], targets: List[float]) -> List[float]:
        """Computes the derivative of the loss function."""
        # This is a placeholder; implement according to your loss function.
        # For cross-entropy with softmax, you would have:
        return [pred - target for pred, target in zip(outputs, targets)]

    def train(self, training_data: List[tuple]):
        """Trains the network on the given data.

        Args:
            training_data: A list of tuples containing input and target data.
        """
        num_samples = len(training_data)
        num_batches = num_samples // self.batch_size

        for epoch in range(self.epochs):
            total_loss = 0
            random.shuffle(training_data)

            for i in range(num_batches):
                batch_data = training_data[i * self.batch_size:(i + 1) * self.batch_size]
                inputs_batch, targets_batch = zip(*batch_data)

                # Forward pass
                outputs_batch = [self.forward(inputs) for inputs in inputs_batch]

                # Calculate loss for this batch
                batch_loss = sum(self.compute_loss(outputs, targets) for outputs, targets in zip(outputs_batch, targets_batch))
                total_loss += batch_loss

                # Backward pass
                self.backward(targets_batch, [output for output in outputs_batch])

            avg_loss = total_loss / num_batches

            # Print loss for this epoch
            print(f"Epoch {epoch + 1}/{self.epochs} complete. Average loss: {avg_loss:.4f}")

    def evaluate(self, test_data: List[tuple]) -> float:
        """Evaluates the network on the test data."""
        inputs_batch, targets_batch = zip(*test_data)
        predictions = [self.forward(inputs) for inputs in inputs_batch]
        accuracy = self.calculate_accuracy(predictions, targets_batch)
        return accuracy

    def predict(self, new_data: List[float]) -> List[float]:
        """Predicts the output for new input data.

        Args:
            new_data: A list of new input data.

        Returns:
            A list of predicted outputs.
        """
        return self.forward(new_data)

    def calculate_accuracy(self, predictions: List[List[float]], targets: List[List[float]]) -> float:
        """Calculates the accuracy of the model."""
        correct_predictions = 0
        for pred, target in zip(predictions, targets):
            predicted_class = np.argmax(pred)
            true_class = np.argmax(target)
            if predicted_class == true_class:
                correct_predictions += 1
        return correct_predictions / len(targets)

In [ ]:
class LossFunction:
    """A utility class for loss functions."""

    @staticmethod
    def cross_entropy(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """Calculates the categorical cross-entropy loss.

        Args:
            predicted_outputs: A list of predicted probabilities for each class.
            actual_outputs: A one-hot encoded list of actual output values.

        Returns:
            The categorical cross-entropy loss.
        """
        # Clip the predicted values to prevent log(0)
        predicted_outputs = np.clip(predicted_outputs, 1e-12, 1 - 1e-12)

        # Calculate the cross-entropy loss
        loss = -sum([actual_output * np.log(predicted_output)
                     for predicted_output, actual_output in zip(predicted_outputs, actual_outputs)])
        return loss / len(predicted_outputs)

    @staticmethod
    def mean_squared_error(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """Calculates the mean squared error (MSE).

        Args:
            predicted_outputs: A list of predicted output values.
            actual_outputs: A list of actual output values.

        Returns:
            The mean squared error.
        """
        squared_errors = [(predicted_output - actual_output) ** 2
                          for predicted_output, actual_output in zip(predicted_outputs, actual_outputs)]
        return sum(squared_errors) / len(squared_errors)


In [ ]:
class WeghitsInitializer:
    """A utility class for weight initialization."""

    @staticmethod
    def weights(num_inputs: int) -> List[float]:
        """Initializes weights with random values uniformly distributed between -0.1 and 0.1.

        Args:
            num_inputs: The number of inputs to the neuron.

        Returns:
            A list of initialized weights.
        """
        return [random.uniform(-0.1, 0.1) for _ in range(num_inputs)]
